In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
from dash.dependencies import Input, Output
import plotly.graph_objs as go

def apokasc():
    filename = "./table5.dat"

    KIC = []
    Teff = []
    M = []
    FeH = []
    R = []
    logg = []
    evo = []
    
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            if "No" not in line:
                KIC.extend([float(line[0:8])])
                Teff.extend([float(line[28:34])])
                M.extend([float(line[117:122])])
                FeH.extend([float(line[41:47])])
                R.extend([float(line[135:141])])
                logg.extend([float(line[154:159])])
                evo.extend([line[94:102].strip()])
                
    KIC = np.asarray(KIC)
    Teff = np.asarray(Teff)
    M = np.asarray(M)
    FeH = np.asarray(FeH)
    R = np.asarray(R)
    logg = np.asarray(logg)
    evo = np.asarray(evo)

    return KIC, Teff, M, FeH, R, logg, evo

KIC, Teff, M, FeH, R, logg, evo = apokasc()

# Build App
app = JupyterDash(__name__)

#server = app.server

app.layout = html.Div([
    html.H1("APOKASC-2 data"),
    html.P("This is an example in JupyterDash. Here, we combine Python with hmtl."),
    dcc.Graph(id='graph'),
    html.Label([
        "Evolutionary stage",
        dcc.Dropdown(
            id='Evolutionary state', clearable=False,
            value='RGB', options=[
                {'label': c, 'value': c}
                for c in ["RC","RGB","AMB"]
            ])
    ]),
    dcc.Markdown('''
    Dash also supports Markdown. Markdown is a simple way to write and format text.
    Here, we use data from [APOKASC-2](https://arxiv.org/abs/1804.09983). For more [examples](https://github.com/plotly/jupyter-dash/blob/master/notebooks/getting_started.ipynb)
    go to the GitHub repo of [JupyterDash](https://github.com/plotly/jupyter-dash). Also,
    check out [this](https://pbpython.com/interactive-dashboards.html#first-option-using-binder).
    ''')    
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("Evolutionary state", "value")]
)
def update_figure(evostate):
    
    mask = evo == evostate
     
    df = {"M":M[mask],
          "Teff":Teff[mask],
          "R":R[mask],
          "FeH":FeH[mask],
          "logg":logg[mask]
    }

    
    Fig = px.scatter(
        df, x="Teff", y="logg", color="M", size="R",
        color_continuous_scale="plasma",
        render_mode="webgl"
    )
    
    Fig.update_layout(
    xaxis_title=r"Effective temperature",
    yaxis_title=r"Surface gravity",
    )
    
    return Fig

#app.run_server()

# Run app and display result inline in the notebook
app.run_server(mode='inline')